In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parameters
f = 5  # Frequency of sine wave (Hz)
A = 1  # Amplitude
fs = 1000  # Sampling frequency
t = np.arange(0, 1, 1/fs)  # Time vector (1 second)

# Generate sine wave
x = A * np.sin(2 * np.pi * f * t)

# Quantization parameters
n_bits = 3  # Number of bits for quantization
L = 2 ** n_bits  # Number of quantization levels

# Quantization process
x_min, x_max = -A, A
delta = (x_max - x_min) / L  # Step size

# Quantize the signal
x_quantized = np.round((x - x_min) / delta) * delta + x_min
x_quantized = np.clip(x_quantized, x_min, x_max)

# Quantization error
q_error = x - x_quantized

# Plot results
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Original sine wave
axes[0].plot(t, x, 'b-', linewidth=1.5, label='Original Sine Wave')
axes[0].set_title(f'Original Sine Wave (f = {f} Hz)')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)
axes[0].legend()

# Quantized signal
axes[1].step(t, x_quantized, 'r-', linewidth=1.5, where='mid', label=f'Quantized ({n_bits}-bit, {L} levels)')
axes[1].plot(t, x, 'b-', alpha=0.3, label='Original')
axes[1].set_title(f'Quantized Signal ({n_bits}-bit Quantization, {L} Levels)')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True)
axes[1].legend()

# Quantization error
axes[2].plot(t, q_error, 'g-', linewidth=1)
axes[2].set_title('Quantization Error')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Error')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print(f"Number of bits: {n_bits}")
print(f"Number of quantization levels: {L}")
print(f"Step size (Δ): {delta:.4f}")
print(f"Max quantization error: {np.max(np.abs(q_error)):.4f}")